
# <svg xmlns="http://www.w3.org/2000/svg" height="24" width="24" viewBox="0 0 512 512"> <path fill="#444444" d="M234.5 5.7c13.9-5 29.1-5 43.1 0l192 68.6C495 83.4 512 107.5 512 134.6V377.4c0 27-17 51.2-42.5 60.3l-192 68.6c-13.9 5-29.1 5-43.1 0l-192-68.6C17 428.6 0 404.5 0 377.4V134.6c0-27 17-51.2 42.5-60.3l192-68.6zM256 66L82.3 128 256 190l173.7-62L256 66zm32 368.6l160-57.1v-188L288 246.6v188z"/> </svg>  **Star Schema SQL Script**


## This is an excerpt of the script I used to create a new database with 7 tables (5 dimension and 2 fact tables) for my Database Design class. The database is named <svg xmlns="http://www.w3.org/2000/svg" height="20" width="17.5" viewBox="0 0 448 512"><!--!Font Awesome Free 6.5.1 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2023 Fonticons, Inc.--><path fill="#4d22b3" d="M448 80v48c0 44.2-100.3 80-224 80S0 172.2 0 128V80C0 35.8 100.3 0 224 0S448 35.8 448 80zM393.2 214.7c20.8-7.4 39.9-16.9 54.8-28.6V288c0 44.2-100.3 80-224 80S0 332.2 0 288V186.1c14.9 11.8 34 21.2 54.8 28.6C99.7 230.7 159.5 240 224 240s124.3-9.3 169.2-25.3zM0 346.1c14.9 11.8 34 21.2 54.8 28.6C99.7 390.7 159.5 400 224 400s124.3-9.3 169.2-25.3c20.8-7.4 39.9-16.9 54.8-28.6V432c0 44.2-100.3 80-224 80S0 476.2 0 432V346.1z"/></svg> <font color='#4d22b3'>**TargetMart**</font>. This notebook shows scripts for 3 of the tables.

  

## <u>Schemas</u>
## <svg xmlns="http://www.w3.org/2000/svg" height="10" width="10" viewBox="0 0 512 512"> <circle cx="256" cy="256" r="256" fill="#0061ff"/> </svg>I created two separate schemas, dim and f, for my dimensions and facts, respectively.

In [ ]:
USE TargetMart;
GO


IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'dim' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA dim AUTHORIZATION dbo;'
END
;

GO

IF NOT EXISTS (SELECT * FROM sys.schemas WHERE name = 'f' ) 
BEGIN
	EXEC sp_executesql N'CREATE SCHEMA f AUTHORIZATION dbo;'
END
;

GO

## <u>Customers Dimension Table</u>
## <svg xmlns="http://www.w3.org/2000/svg" height="10" width="10" viewBox="0 0 512 512"><!--!Font Awesome Free 6.5.1 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2023 Fonticons, Inc.--><path fill="#0061ff" d="M256 512A256 256 0 1 0 256 0a256 256 0 1 0 0 512z"/></svg> This table has a surrogate key (pkCustomer) and a natural key (CustomerID). The load date is also included to keep track of when new records are added to the table. </span>

In [ ]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Customers')

BEGIN
	CREATE TABLE dim.Customers(
        pkCustomer int IDENTITY(1000,1) NOT NULL,
        CustomerID nvarchar(5) NOT NULL,
        Customer nvarchar(40) NOT NULL,
        City nvarchar(15) NULL,
        Country nvarchar(15) NULL,
        LoadDate DATE NOT NULL,
    );

	ALTER TABLE dim.Customers
	ADD CONSTRAINT PK_Customers_LUP PRIMARY KEY(pkCustomer);

	ALTER TABLE dim.Customers
    ADD CONSTRAINT UC_Customers_ID UNIQUE (CustomerID);

END
;
GO


## <u>Calendar Dimension Table</u>
## <svg xmlns="http://www.w3.org/2000/svg" height="10" width="10" viewBox="0 0 512 512"><!--!Font Awesome Free 6.5.1 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2023 Fonticons, Inc.--><path fill="#0061ff" d="M256 512A256 256 0 1 0 256 0a256 256 0 1 0 0 512z"/></svg> The primary key will be in the format of YYYYMMDD. </span>

In [ ]:

IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'dim' AND TABLE_NAME = 'Calendar')

BEGIN
    CREATE TABLE dim.Calendar(
        pkCalendar INT NOT NULL,
        DateValue DATE NOT NULL,
        Year INT NOT NULL,
        Quarter INT NOT NULL,
        Month INT NOT NULL,
        MonthName VARCHAR(10) NOT NULL,
        MonthShort VARCHAR(3) NOT NULL,
        WeekofYear INT NOT NULL,
        DayOfMonth INT NOT NULL,
        DayName VARCHAR(10) NOT NULL,
        DayShort VARCHAR(3) NOT NULL,
        Weekday VARCHAR(3) NOT NULL
    );

	ALTER TABLE dim.Calendar
	ADD CONSTRAINT PK_Calendar PRIMARY KEY(pkCalendar);

	ALTER TABLE dim.Calendar
    ADD CONSTRAINT UC_Calendar UNIQUE (DateValue);

END
;
GO

## <u>Order Performance Fact Table</u>
## <svg xmlns="http://www.w3.org/2000/svg" height="10" width="10" viewBox="0 0 512 512"><!--!Font Awesome Free 6.5.1 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2023 Fonticons, Inc.--><path fill="#0061ff" d="M256 512A256 256 0 1 0 256 0a256 256 0 1 0 0 512z"/></svg> There is no primary key and there are 4 foreign keys. The relationships are established to 4 of the dimension tables.</span>

In [ ]:
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'f' AND TABLE_NAME = 'OrderPerf')

BEGIN
    CREATE TABLE f.OrderPerf(
        OrderID int NOT NULL,
        fkCalendar int NOT NULL,
        fkCustomer int NOT NULL,
        fkEmployee int NULL,
        fkShipper int NULL,
        Freight money NOT NULL,
        DaysToShip int NULL,
        DaysTilRequired int NOT NULL,
        OrderCount bit NOT NULL
    );

    ALTER TABLE f.OrderPerf
    ADD CONSTRAINT PK_ORD PRIMARY KEY(OrderID);

    ALTER TABLE f.OrderPerf
    ADD CONSTRAINT FK_ORDtoCAL
        FOREIGN KEY (fkCalendar)
        REFERENCES dim.Calendar(pkCalendar);

    ALTER TABLE f.OrderPerf
    ADD CONSTRAINT FK_ORDtoCUST
        FOREIGN KEY (fkCustomer)
        REFERENCES dim.Customers(pkCustomer);

    ALTER TABLE f.OrderPerf
    ADD CONSTRAINT FK_ORDtoEMP
        FOREIGN KEY (fkEmployee)
        REFERENCES dim.Employees(pkEmployeeID);

    ALTER TABLE f.OrderPerf
    ADD CONSTRAINT FK_ORDtoSHIP
        FOREIGN KEY (fkShipper)
        REFERENCES dim.Shipper(pkShipperID);

END
;
GO

## <svg xmlns="http://www.w3.org/2000/svg" height="20" width="22.5" viewBox="0 0 576 512"><!--!Font Awesome Free 6.5.1 by @fontawesome - https://fontawesome.com License - https://fontawesome.com/license/free Copyright 2023 Fonticons, Inc.--><path fill="#444444" d="M0 80C0 53.5 21.5 32 48 32h96c26.5 0 48 21.5 48 48V96H384V80c0-26.5 21.5-48 48-48h96c26.5 0 48 21.5 48 48v96c0 26.5-21.5 48-48 48H432c-26.5 0-48-21.5-48-48V160H192v16c0 1.7-.1 3.4-.3 5L272 288h96c26.5 0 48 21.5 48 48v96c0 26.5-21.5 48-48 48H272c-26.5 0-48-21.5-48-48V336c0-1.7 .1-3.4 .3-5L144 224H48c-26.5 0-48-21.5-48-48V80z"/></svg> _The entity relationship diagram for this database is uploaded in my repository._